# Multi modal model

In [ ]:
%pip install yfinance

In [7]:
import yfinance as yf
import numpy as np

import pandas as pd
import glob
import re
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Computing\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [8]:
#now that I have the whole string for the day, I need to do processing first, before spliting them into chunks. 
words = set(nltk.corpus.words.words())
#I need to add in some crypto currency specific words. like Etherum.
#bascailly all the symbols of crypto currencies, I should add a set here. 
cryptowords=(['crypto','cryptocurrency','bitcoin','btc','etherum','eos','eth','xrp','ltc'])
words.update(cryptowords)

exclusionli=['up','so','if','go'] #keep in view


def processTweet(tweetFeed):
    tweetFeed = tweetFeed.lower()
    #Convert www.* or https?://* pic.twitter.com* to space
    tweetFeed = re.sub('(https:\/\/www\.[\s][\a-zA-Z]*)|(http:\/\/www\.[\s][\a-zA-Z]*)|(https:\/\/[\s][\a-zA-Z]*)|(http:\/\/[\s][\a-zA-Z]*)|(www\.[\s][\a-zA-Z]*)|(https:\/\/[\a-zA-Z]*)|(http:\/\/[\a-zA-Z]*)',' ',tweetFeed)
    tweetFeed = re.sub('pic\.[^\s]*',' ',tweetFeed)
    #removw @sth and #sth
    tweetFeed = re.sub('(@\ [^\s]*)|(#\ [^\s]*)', ' ', tweetFeed)
    #remove rt
    tweetFeed = re.sub('rt[\s]','',tweetFeed)

    tweetFeed = " ".join(w for w in nltk.wordpunct_tokenize(tweetFeed) if w.lower() in words)# or not w.isalpha())#this part takes in number as True.
    #I'm not so sure about this language filter, but let's just roll with it first. 
    tweetFeed = re.sub("[^A-Za-z']+", ' ', tweetFeed)

    #filter out all the token left with len of 1. possibaly 2, defind a exclusion list: up, if, so, 
    tweetFeed = " ".join(w for w in nltk.wordpunct_tokenize(tweetFeed) if len(w) >1 )
    return tweetFeed

In [9]:
#now let's try to split the text into smaller chunks, which is just taking the tutorial's code
def get_split(text1,length=200, overlap=50):
  l_total = []
  l_parcial = []
  if len(text1.split())//(length-overlap) >0:
    n = len(text1.split())//(length-overlap)
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      l_parcial = text1.split()[:length]
      l_total.append(" ".join(l_parcial))
    else:
      l_parcial = text1.split()[w*(length-overlap):w*(length-overlap) + length]
      l_total.append(" ".join(l_parcial))
  return l_total

In [12]:
#next, tie them up, read in csv, combine into chunks, take the dates as the price? 
path = '../archive/data/tweets'
all_files = glob.glob(path+'/*.csv')

#let me try to use list1 to store string and list 2 to store dates
li1=[]
li2=[]
for file in all_files:
  df=pd.read_csv(file,sep=',',index_col=0)
  df=df['text'].to_frame()
  rawtxt=[]
  for row in df.text:
    rawtxt.append(row)
  string = " ".join(rawtxt)

  date= file[-14:-4]
  
  li1.append(string)
  li2.append(date)

data=pd.DataFrame({'date':li2, 'text':li1})



AttributeError: 'DataFrame' object has no attribute 'text'

In [3]:
start_date ='2018-12-28'
end_date = '2019-04-02'
price = yf.download("BTC-USD", start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [6]:
price.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-28,3653.131836,3956.135986,3642.632080,3923.918701,3923.918701,5631554348
2018-12-29,3932.491699,3963.758789,3820.408691,3820.408691,3820.408691,4991655917
2018-12-30,3822.384766,3901.908936,3797.219238,3865.952637,3865.952637,4770578575
2018-12-31,3866.839111,3868.742920,3725.867432,3742.700439,3742.700439,4661840806
2019-01-01,3746.713379,3850.913818,3707.231201,3843.520020,3843.520020,4324200990
